# Levelized Costs of Hydrogen
The LCoE is often used to compare different energy–generating technologies [70], [86], [111], [122]. This approach is applied by this master thesis to compute the LCoH2 issued by the PtH2 unit. The LCoH2 are calculated by equation 2.11 which includes the annual Capital Expenditures (CapEx), the annual Operating Expenditures (OpEx) and the annual amount of hydrogen produced mH2,a. The CapEx are considered according to the annuity method based on the norm DIN 2067 issued by the “Verein Deutsche Ingenieure” [142].

$LCoH_2 = \frac{CapEx_a + OpEx_a}{m_{H_2,a}}$
\begin{align*}
CapEx_a & : \text{Annual Capital Expenditures} \\
OpEx_a & : \text{Annual Operating Expenditures} \\
m_{H_2,a} & : \text{Annual mass of produced hydrogen}
\end{align*}


$\text{CapEx}_a = \sum \left( (Inv_{\text{comp}} + Inv_{\text{comp},re} - Value_{\text{comp},Residual}) \cdot A_r \right)$
\begin{align*}
Inv, Inv_re & : \text{Specific investment, specific reinvestment}\\
Value_Residual  & : \text{Residual value after project lifetime}\\
A_r & : \text{ Annuity factor}
\end{align*}

In [19]:
import pandas as pd
#data input
electrolyzers_df = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/input/2016/electrolyzers.csv')
economic_assumption_df = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/Data/economic_assumptions.csv')
display(electrolyzers_df)
display(economic_assumption_df)


,name,technology,minLoad,maxLoad,installedCapacity,effElec,minDowntime,minRuntime,coldStartUpCost,maxAllowedColdStartups,standbyCons,comprCons,maxSOC,company
0,Iron_steel,PEM,0.05,1.2,11850,0.63,0.5,1,50,3000,0.0005,0.0012,100000,Iron_steel


,technology,CAPEX,stack_replacement,OPEX,discount_rate,lifetime,stack_lifetime,scaling_factor
0,PEM,1078,25.0,3,7,20,10.0,-0.1976
1,AEL,902,25.0,3,7,20,10.0,-0.1976
2,Storage,530,NaN,2,7,30,NaN,NaN
3,Compressor,2500,NaN,5,7,15,NaN,NaN


### CAPEX Calculation
Calculating Electrolyzer CAPEX

In [21]:

def calculate_elec_CAPEX(technology, capacity):
    if technology == 'PEM':
        CAPEX = (capacity * 1078)**(-0.1976) + (capacity * 1078*0.25)
    if technology == 'AEL':
        CAPEX = (capacity * 902)**(-0.1976) + (capacity * 902*0.25)
    return CAPEX

iron_steel_elec_CAPEX = calculate_elec_CAPEX(technology='PEM', capacity = electrolyzers_df.loc[0, 'installedCapacity'])
iron_steel_elec_CAPEX

3193575.0394264604

#### Compressor CAPEX

#### Storage CAPEX

### OPEX calculation

In [ ]:
EOM_prices_df = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/output/2016/EOM_Prices.csv')
iron_steel_df = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/output/2016/Elec_capacities/Elec_x_optimizedBidAmount.csv')
